# OPL model:  scheduling.mod  + supplydata.dat

##  Converted to a python gurobi model, basic steps:

* Convert OPL dat file to a JSON file  
* Copy OPL mod file to a python editor and ...
* Change OPL parameters and sets to python structures
+ Read in the JSON file
+ Use JSON data to initialize more python data
+ Create gurobi variables for each OPL dvar
+ Convert OPL objective function to gurobi python
+ Convert each OPL constraint to python
+ Solve the model

General notes

#### Basic imports for python and Gurobi:

In [ ]:
from __future__ import print_function
import itertools
from collections import namedtuple
import json
import pandas as pd
import gurobipy as grb
GRB = grb.GRB

### Note the difference between OPL ranges and Python ranges:

OPL Code:

    int Nperiods = 4;
    range Periods   = 1..Nperiods;
    range ZPeriods  = 0..Nperiods;
    range Products  = 1..2;
    range Factories = 1..2;
    range Stages    = 2..3;

In [ ]:
Nperiods = 4
Periods = list(range(1, Nperiods+1))
ZPeriods = list(range(0, Nperiods+1))
Products = ['1','2']
Factories = ['1','2']
Stages = [2,3]

Changes going from OPL to Python:

* OPL uses "..";  Python uses "range()" method
* Python is not a typed language, so OPL's "int" and "float" are discarded.
* Some ranges are converted to strings, since they are used as keys.
* We leave other ranges as numbers, because computations are done with time and stages in this model.

#### Next step is to convert the OPL dat file to a JSON file, using our opl2json utility:

Here is a fragment of the OPL .dat file: 

    ProdCost = #[
                  1: [ 1    0.5 ]    // Factory 1
                  2: [ 0.5  0.25 ]   // Factory 2
                ]#;
                
    MovCost_FDC = [ 0.1 0.3 ];           

and the JSON file created from OPL dat file by running our opl2json utility:

    "ProdCost" : {
    "1": [ 1, 0.5 ],
    "2": [ 0.5, 0.25 ]
    },

    "MovCost_FDC" : [ 0.1, 0.3 ],


JSON = Java Script Object Notation

JSON keys are in quotes.

### Now we are ready to read in the OPL data into our python program

All the data from the original .dat file should now be in the JSON file.

Use the json python package to read in a JSON file:

In [ ]:
with open('supplydata1.json') as f:
    d = json.loads(f.read())

The python dictionary "d" now contains all the data from the OPL .dat file:

    {
     u'ProdCost': {u'1': [1, 0.5], u'2': [0.5, 0.25]},
     u'MovCost_FDC': [0.1, 0.3],
     ...

### This next part of converting from OPL to python is critical.  If the data from OPL is converted correctly, then the rest of the model conversion is easy and straightforward.

For example, if the OPL model uses a data structure like this: 

y[i,2,j,t-1]  &nbsp;&nbsp;&nbsp;&nbsp;    <---- Note 4 indices

then we want to create a python data structure called "y" that has four indices. (Use a python dictionary with a 4-tuple key) and in this case the second and fourth indices need to be numeric.

If OPL has: 

bnd.v   &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;         <---- Note use of "dot" notation

then python variable "bnd" needs to support the dot notation. (Use namedtuple or pandas series)

From the json data, we recreate each OPL param and set with python data structures:

We create a python variable to match every OPL variable, with data from the python dictionary "d":

In [ ]:
RequiredLotSize = d['RequiredLotSize']
Demand = {(i, j, k): d['Demand'][str(i)][j][k-1] for i in Stages 
                                                  for j in Products 
                                                  for k in Periods}
ProdCost = {(i, j): d['ProdCost'][i][idx] for i in Factories 
                                        for idx, j in enumerate(Products)}
ProdTime = {(i, j): d['ProdTime'][i][idx] for i in Factories 
                                        for idx, j in enumerate(Products)}
HoldCost = d['HoldCost']
MovCost_FDC = {j: d['MovCost_FDC'][idx] for idx, j in enumerate(Factories)}
MovCost_FC = {j: d['MovCost_FC'][idx] for idx, j in enumerate(Factories)}
MovCost_DCC = d['MovCost_DCC']
TardCost_DC = {j: d['TardCost_DC'][idx] for idx, j in enumerate(Products)}
TardCost_C = {j: d['TardCost_C'][idx] for idx, j in enumerate(Products)}
Penalty = d['Penalty']
bnd = namedtuple('bnd','type F S P v')
bnds = [bnd(*mybnd) for mybnd in d['bnds']]

### With python data taking the same parameters as the OPL, we just convert OPL syntax to python syntax:

OPL code:

    float MaxProduction = 
            sum (k in Stages, j in Products, t in Periods)  Demand[k,j,t];

    int  LotSize = maxl(RequiredLotSize, 1);

In [ ]:
MaxProduction = sum ([Demand[k,j,t] for k in Stages 
                                    for j in Products 
                                    for t in Periods]) 

LotSize = max(RequiredLotSize, 1);

Note the difference in syntax: python uses the word "for", and doesn't use commas.

### For creating model variables, we use a couple helper functions.



In [ ]:
def make_name(*args):
    return".".join(str(arg) for arg in args)

def get_vars(model, name, *indices, **kwargs):
    """
    returns a dictionary of variables,
    for  all combinations of indices
    """
    return pd.Series({keys: model.addVar(name=make_name(name, *keys), **kwargs)
                      for keys in itertools.product(*indices)})

### Convert OPL dvar's to gurobi variables:

OPL code:

    dvar float+    x[Factories, Products, Periods];
    dvar int+      y[Factories, Stages, Products, Periods] in 0..maxint;
    dvar float+    z[Products, Periods];
    dvar float+    q2[Products, ZPeriods];
    dvar float+    v2[Products, Periods];
    dvar float+    v3[Products, ZPeriods];
    dvar boolean   yb[bnds, Periods];
    dvar boolean   q2b[Products, Periods];
    dvar boolean   v2b[Products, Periods];
    dvar boolean   v3b[Products, Periods];
    dvar int       xlots[Factories, Products, Periods] in 0 .. ftoi(MaxProduction/LotSize);

We create the gurobi model, then for each OPL dvar, we create a corresponding gurobi variable, using the get_vars helper function:

In [ ]:

model = grb.Model()
x = get_vars(model, 'x', Factories, Products, Periods)
y = get_vars(model, 'y', Factories, Stages, Products, Periods, 
             vtype=GRB.INTEGER)
z = get_vars(model, 'z', Products, Periods)
q2 = get_vars(model, 'q2', Products, ZPeriods)
v2 = get_vars(model, 'v2', Products, Periods)
v3 = get_vars(model, 'v3', Products, ZPeriods)
yb = get_vars(model, 'yb', bnds, Periods, vtype=GRB.BINARY)

q2b = get_vars(model, 'q2b', Products, Periods, vtype=GRB.BINARY)
v2b = get_vars(model, 'v2b', Products, Periods, vtype=GRB.BINARY)
v3b = get_vars(model, 'v3b', Products, Periods, vtype=GRB.BINARY)
xlots = get_vars(model, 'xlots', Factories, Products, Periods,
                 vtype=GRB.INTEGER, ub=int(MaxProduction/LotSize))
model.update()

After adding the variables, be sure to update the gurobi model so that you can add the objective function and constraints!

### Setting the objective function

Use the gurobi method setObjective().

The OPL sum needs to be converted to Gurobi quicksum, rather than just python sum, to be able to handle gurobi variables:

OPL Objective:

    minimize
       sum(t in Periods, j in Products, i in Factories) ProdCost[i,j]*x[i,j,t] +
       sum(t in Periods, j in Products, i in Factories) MovCost_FDC[i]*y[i,2,j,t] +
       sum(t in Periods, j in Products, i in Factories) MovCost_FC[i]*y[i,3,j,t] +
       sum(t in Periods, j in Products)                 MovCost_DCC*z[j,t] +
       sum(t in Periods, j in Products)                 HoldCost*q2[j,t] +
       sum(t in 1 .. Nperiods-1, j in Products)         7*TardCost_DC[j]*v2[j,t] +
       sum(t in 1 .. Nperiods-1, j in Products)         7*TardCost_C[j]*v3[j,t] +
       sum(j in Products)                  Penalty*(v2[j,Nperiods] + v3[j,Nperiods]);

In [ ]:
model.setObjective(
        grb.quicksum([ProdCost[i,j]*x[i,j,t] for i in Factories 
                                             for j in Products 
                                             for t in Periods]) +
        grb.quicksum([MovCost_FDC[i]*y[i,2,j,t] for i in Factories 
                                                for j in Products 
                                                for t in Periods]) +
        grb.quicksum([MovCost_FC[i]*y[i,3,j,t] for i in Factories 
                                               for j in Products 
                                               for t in Periods]) +
        grb.quicksum([MovCost_DCC*z[j,t] for j in Products 
                                         for t in Periods]) +
        grb.quicksum([HoldCost*q2[j,t] for j in Products 
                                       for t in Periods]) +
        grb.quicksum([7*TardCost_DC[j]*v2[j,t] for j in Products 
                                               for t in Periods[:-1]]) +
        grb.quicksum([7*TardCost_C[j]*v3[j,t] for j in Products 
                                              for t in Periods[:-1]]) +
        grb.quicksum([Penalty*(v2[j,Nperiods] + v3[j,Nperiods]) 
                                              for j in Products]),  
    sense=GRB.MINIMIZE)

The expression inside quicksum is exactly the same as the OPL expression.

### Helper function for constraints:

In [ ]:
def addConstr(expr):
    model.addConstr(expr)

The remaining conversion is a one-to-one correspondance of each OPL statement to a Python statement.

Most of the OPL expressions are identical to the python expressions.

OPL code:

    subject to
    {
       if (RequiredLotSize > 0)
          forall (i in Factories, j in Products, t in Periods)
             x[i,j,t] == RequiredLotSize*xlots[i,j,t];
       else
          forall (i in Factories, j in Products, t in Periods)
             xlots[i,j,t] == 0;

In [ ]:
if RequiredLotSize > 0:    
    [addConstr(x[i,j,t] == RequiredLotSize*xlots[i,j,t]) 
         for i in Factories for j in Products for t in Periods]
else:
    [addConstr(xlots[i,j,t] == 0) 
         for i in Factories for j in Products for t in Periods]     

Using python list comprehension here makes the correspondance clear between OPL and Python.

OPL code:

       // Production Capacity
       forall (t in Periods, i in Factories)
          sum (j in Products) ProdTime[i,j]*x[i,j,t] <= 168;

In [ ]:
[addConstr(grb.quicksum([ProdTime[i,j]*x[i,j,t] for j in Products]) <= 168) 
            for t in Periods for i in Factories]

Use gurobi's quicksum method if we are summing gurobi variables!

OPL Code:
    
       // Disjunctive bounds for lower bounds, or specified upper bounds.
       forall (b in bnds, t in Periods) {
          if (b.type == "L") {
             y[b.F, b.S, b.P, t] >= yb[b,t]*b.v;
             y[b.F, b.S, b.P, t] <= yb[b,t]*MaxProduction;
          }
          if (b.type == "U") {
             y[b.F, b.S, b.P, t] <= b.v;
             yb[b,t] == 0;
          }
       }

In [ ]:
[(addConstr(y[b.F, b.S, b.P, t] >= yb[b,t]*b.v), 
  addConstr(y[b.F, b.S, b.P, t] <= yb[b,t]*MaxProduction))
     for b in bnds for t in Periods if b.type == 'L']

[(addConstr(y[b.F, b.S, b.P, t] <= b.v), 
  addConstr(yb[b,t] == 0))
    for b in bnds for t in Periods if b.type == 'U']

We could have used python "for" statements here to mimic the OPL more closely.

OPL Code:

       // Transportation constraints
       forall (t in Periods, j in Products, i in Factories)
          sum (l in Stages) y[i,l,j,t] == x[i,j,t];

       forall (t in Periods, j in Products : t < 4)
          sum (i in Factories) y[i,3,j,t] + z[j,t] <= Demand[3,j,t+1] + v3[j,t];

       forall (j in Products)
          z[j,1] <= q2[j,0];  // RHS should be max(0,q2[j,0]), but this is the same

       forall (t in Periods, j in Products : t > 1)
          z[j,t] <= q2[j,t-1] + sum (i in Factories) y[i,2,j,t-1];

In [ ]:
[addConstr(grb.quicksum([y[i,l,j,t] for l in Stages]) == x[i,j,t]) 
           for t in Periods for j in Products for i in Factories]

[addConstr(grb.quicksum([y[i,3,j,t] for i in Factories]) + z[j,t] <= 
                       Demand[3,j,t+1] + v3[j,t]) 
           for t in Periods[:-1] for j in Products]
# or       for t in Periods for j in Products if t < 4]

[addConstr(z[j,1] <= q2[j,0]) 
           for j in Products]

[addConstr(z[j,t] <= q2[j,t-1] + grb.quicksum([y[i,2,j,t-1] for i in Factories])) 
           for t in Periods for j in Products if t > 1]

OPL Code:

    // Storage constraints
       forall (j in Products) {
          q2[j,1] >= q2[j,0] - Demand[2,j,1] - z[j,1];
          q2[j,1] <= q2b[j,1]*MaxProduction;
          q2[j,1] <= q2[j,0] - Demand[2,j,1] - z[j,1] + MaxProduction - 
                                                  MaxProduction*q2b[j,1];
          forall (t in Periods : t > 1) {
             q2[j,t] >= q2[j,t-1] + sum (i in Factories) y[i,2,j,t-1] - 
                                        Demand[2,j,t] - z[j,t] - v2[j,t-1];
             q2[j,t] <= q2b[j,t]*MaxProduction;
             q2[j,t] <= q2[j,t-1] + sum (i in Factories) y[i,2,j,t-1] - 
                  Demand[2,j,t] - z[j,t] -v2[j,t-1] + MaxProduction*(1.0-q2b[j,t]);
          }

In [ ]:
for j in Products:
    addConstr(q2[j,1] >= q2[j,0] - Demand[2,j,1] - z[j,1])
    addConstr(q2[j,1] <= q2b[j,1]*MaxProduction)
    addConstr(q2[j,1] <= q2[j,0] - Demand[2,j,1] - z[j,1] + MaxProduction - 
                                                  MaxProduction*q2b[j,1])
    for t in Periods:
        if t > 1:
            addConstr(q2[j,t] >= q2[j,t-1] + grb.quicksum([y[i,2,j,t-1] 
                    for i in Factories])  - Demand[2,j,t] - z[j,t] - v2[j,t-1] )
            addConstr(q2[j,t] <= q2b[j,t]*MaxProduction)
            addConstr(q2[j,t] <= q2[j,t-1] + grb.quicksum([y[i,2,j,t-1] 
                                                        for i in Factories]) 
             - Demand[2,j,t] - z[j,t] -v2[j,t-1] + MaxProduction*(1.0-q2b[j,t]))
        

OPL Code:

       // Tardy and jobs not delivered
       forall (j in Products) {
          v2[j,1] >= Demand[2,j,1] - q2[j,0];
          v2[j,1] <= v2b[j,1]*MaxProduction;
          v2[j,1] <= Demand[2,j,1] - q2[j,0] + MaxProduction*(1.0-v2b[j,1]);

          v3[j,1] >= Demand[3,j,1];
          v3[j,1] <= v3b[j,1]*MaxProduction;
          v3[j,1] <= Demand[3,j,1] + MaxProduction*(1.0-v3b[j,1]);

          forall (t in Periods : t > 1) {
             v2[j,t] >= Demand[2,j,t] + v2[j,t-1] + z[j,t] - q2[j,t] - 
                                             sum(i in Factories) y[i,2,j,t-1];
             v2[j,t] <= v2b[j,t]*MaxProduction;
             v2[j,t] <= Demand[2,j,t] + v2[j,t-1] + z[j,t] - q2[j,t] - 
                     sum(i in Factories) y[i,2,j,t-1] + MaxProduction*(1.0-v2b[j,t]);

             v3[j,t] >= Demand[3,j,t] + v3[j,t-1] - z[j,t-1] - 
                                                 sum(i in Factories) y[i,3,j,t-1];
             v3[j,t] <= MaxProduction*v3b[j,t];
             v3[j,t] <= Demand[3,j,t] + v3[j,t-1] - z[j,t-1] - 
                     sum(i in Factories) y[i,3,j,t-1] + MaxProduction*(1.0-v3b[j,t]);
          }
       }


In [ ]:
for j in Products:
    addConstr(v2[j,1] >= Demand[2,j,1] - q2[j,0])
    addConstr(v2[j,1] <= v2b[j,1]*MaxProduction)
    addConstr(v2[j,1] <= Demand[2,j,1] - q2[j,0] + MaxProduction*(1.0-v2b[j,1]))
    
    addConstr(v3[j,1] >= Demand[3,j,1])
    addConstr(v3[j,1] <= v3b[j,1]*MaxProduction)
    addConstr(v3[j,1] <= Demand[3,j,1] + MaxProduction*(1.0-v3b[j,1]))
    
    for t in Periods:
        if t > 1:
            addConstr(v2[j,t] >= Demand[2,j,t] + v2[j,t-1] + z[j,t] - q2[j,t] - 
                      grb.quicksum([y[i,2,j,t-1] for i in Factories]))
            addConstr(v2[j,t] <= v2b[j,t]*MaxProduction)
            addConstr(v2[j,t] <= Demand[2,j,t] + v2[j,t-1] + z[j,t] - q2[j,t] - 
                      grb.quicksum([y[i,2,j,t-1] for i in Factories]) + 
                                                  MaxProduction*(1.0-v2b[j,t]))
            
            addConstr(v3[j,t] >= Demand[3,j,t] + v3[j,t-1] - z[j,t-1] - 
                                  grb.quicksum([y[i,3,j,t-1] for i in Factories]) )
            addConstr(v3[j,t] <= MaxProduction*v3b[j,t])
            addConstr(v3[j,t] <= Demand[3,j,t] + v3[j,t-1] - z[j,t-1] - 
                              grb.quicksum([y[i,3,j,t-1] for i in Factories]) + 
                              MaxProduction*(1.0-v3b[j,t]))            

OPL Code:

       // Boundary condition
       forall (j in Products) {
          v3[j,0] == 0;
          q2[j,0] == 0;
       }

In [ ]:
[(addConstr(v3[j,0] == 0),
  addConstr(q2[j,0] == 0) )
     for j in Products]

### After the model is created, we can write it out as an .lp file, and run the model:

In [ ]:
model.update()
model.write('nyu_scheduling.lp')
model.optimize()

gurobi.log:

    Optimize a model with 146 rows, 132 columns and 460 nonzeros
    Coefficient statistics:
      Matrix range    [1e-03, 3e+05]
      Objective range [5e-02, 1e+03]
      Bounds range    [1e+00, 3e+05]
      RHS range       [2e+02, 4e+05]
    Found heuristic solution: objective 4.30093e+07
    Presolve removed 68 rows and 63 columns
    Presolve time: 0.00s
    Presolved: 78 rows, 69 columns, 317 nonzeros
    Variable types: 24 continuous, 45 integer (21 binary)

    Root relaxation: objective 2.893794e+06, 21 iterations, 0.00 seconds

        Nodes    |    Current Node    |     Objective Bounds      |     Work
     Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

         0     0 2893793.89    0    8 4.3009e+07 2893793.89  93.3%     -    0s
    H    0     0                    3061001.1000 2893793.89  5.46%     -    0s
    H    0     0                    3013257.0500 2893793.89  3.96%     -    0s
    H    0     0                    3012125.0000 2893793.89  3.93%     -    0s
         0     0 2919815.79    0    9 3012125.00 2919815.79  3.06%     -    0s
         0     0 2921229.83    0   11 3012125.00 2921229.83  3.02%     -    0s
    H    0     0                    3007493.7000 2921229.83  2.87%     -    0s
         0     0 2953729.12    0   10 3007493.70 2953729.12  1.79%     -    0s
    H    0     0                    3007491.6500 2953729.12  1.79%     -    0s
         0     0 2953818.76    0   12 3007491.65 2953818.76  1.78%     -    0s
         0     0 2967327.64    0   16 3007491.65 2967327.64  1.34%     -    0s
         0     0 2967580.55    0   15 3007491.65 2967580.55  1.33%     -    0s
         0     0 2978737.35    0   12 3007491.65 2978737.35  0.96%     -    0s
         0     0 2981486.93    0   11 3007491.65 2981486.93  0.86%     -    0s
         0     0 2982023.12    0   12 3007491.65 2982023.12  0.85%     -    0s
         0     0 2982023.12    0   12 3007491.65 2982023.12  0.85%     -    0s
    H    0     0                    3006991.7000 2982023.12  0.83%     -    0s
         0     2 2982023.12    0   12 3006991.70 2982023.12  0.83%     -    0s
    H 1942    28                    3004950.6500 2995431.37  0.32%   1.0    0s
    * 1943    20               6    3004950.2000 2995431.37  0.32%   1.0    0s

    Cutting planes:
      Gomory: 4
      MIR: 11
      Flow cover: 3

    Explored 2793 nodes (3337 simplex iterations) in 0.10 seconds
    Thread count was 8 (of 8 available processors)

    Optimal solution found (tolerance 1.00e-04)
    Best objective 3.004950200000e+06, best bound 3.004950200000e+06, gap 0.0%

oplrun scheduling.mod supplydata.dat

    IBM ILOG CPLEX Optimization Studio Community Edition.  The CPLEX Optimizers will solve problems up to 1000 variables and 1000 constraints.
    CP Optimizer (Community Edition) solves problems with a search space of up to 2^1000

    <<< setup


    <<< generate

    Tried aggregator 3 times.
    MIP Presolve eliminated 59 rows and 46 columns.
    MIP Presolve modified 59 coefficients.
    Aggregator did 17 substitutions.
    Reduced MIP has 78 rows, 69 columns, and 316 nonzeros.
    Reduced MIP has 21 binaries, 24 generals, 0 SOSs, and 0 indicators.
    Presolve time = 0.00 sec. (0.55 ticks)
    Probing fixed 0 vars, tightened 1 bounds.
    Probing time = 0.00 sec. (0.02 ticks)
    Tried aggregator 1 time.
    Reduced MIP has 78 rows, 69 columns, and 316 nonzeros.
    Reduced MIP has 21 binaries, 24 generals, 0 SOSs, and 0 indicators.
    Presolve time = 0.00 sec. (0.14 ticks)
    Probing time = 0.00 sec. (0.02 ticks)
    MIP emphasis: balance optimality and feasibility.
    MIP search method: dynamic search.
    Parallel mode: deterministic, using up to 8 threads.
    Root relaxation solution time = 0.00 sec. (0.18 ticks)

            Nodes                                         Cuts/
       Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

          0     0  2893793.8910     8                2893793.8910       15
          0     0  2940205.6277     9                    Cuts: 23       30
          0     0  2959906.2928     8                    Cuts: 14       41
          0     0  2962498.3313     7                     Cuts: 8       45
          0     0  2964612.9074     8                     Cuts: 6       51
          0     0  2964819.8040    10                  MIRcuts: 4       55
          0     0  2966480.5889     7                  MIRcuts: 3       57
          0     0  2966537.1545     7                  MIRcuts: 2       58
          0     0  2967783.3667     6                  MIRcuts: 1       59
          0     0  2967898.8429     7                     Cuts: 4       61
    *     0+    0                      3126650.0000  2967898.8429             5.08%
    *     0+    0                      3004950.2000  2967898.8429             1.23%

    Flow cuts applied:  4
    Mixed integer rounding cuts applied:  7
    Lift and project cuts applied:  1
    Gomory fractional cuts applied:  7

    Root node processing (before b&c):
      Real time             =    0.01 sec. (5.61 ticks)
    Parallel b&c, 8 threads:
      Real time             =    0.00 sec. (0.00 ticks)
      Sync time (average)   =    0.00 sec.
      Wait time (average)   =    0.00 sec.
                              ------------
    Total (root+branch&cut) =    0.01 sec. (5.61 ticks)

    <<< solve


    OBJECTIVE: 3004950


OPL Display solution:
    
    // Report solution values
    float TotProdCost    = sum (t in Periods, j in Products, i in Factories)
                ProdCost[i,j]*x[i,j,t];
    float TotMovCost_FDC = sum (t in Periods, j in Products, i in Factories) 
                MovCost_FDC[i]*y[i,2,j,t];
    float TotMovCost_FC  = sum (t in Periods, j in Products, i in Factories) 
                MovCost_FC[i]*y[i,3,j,t];
    float TotMovCost_DCC = sum (t in Periods, j in Products)                 
                MovCost_DCC*z[j,t];
    float TotHoldCost    = sum (t in Periods, j in Products)                 
                HoldCost*q2[j,t];
    float TotTardCost_DC = sum (t in 1..Nperiods-1, j in Products)       
                7*TardCost_DC[j]*v2[j,t];
    float TotTardCost_C  = sum (t in 1..Nperiods-1, j in Products)       
                7*TardCost_C[j]*v3[j,t];
    float TotPenalty     = sum (j in Products)                               
                Penalty*(v2[j,Nperiods] + v3[j,Nperiods]);

    float TotY[i in Factories, k in Stages, t in Periods] = 
                sum (j in Products) y[i,k,j,t];

    execute DISPLAY {
       writeln("Total Production Cost:  ", TotProdCost);
       writeln("Total Move Cost FDC:    ", TotMovCost_FDC);
       writeln("Total Move Cost FC:     ", TotMovCost_FC);
       writeln("Total Move Cost DCC:    ", TotMovCost_DCC);
       writeln("Total Hold Cost:        ", TotHoldCost);
       writeln("Total Tardy Cost DC:    ", TotTardCost_DC);
       writeln("Total Tardy Cost C:     ", TotTardCost_C);
       writeln("Total Penalty:          ", TotPenalty);

       for (i in Factories)
          for (k in Stages)
             for (t in Periods)
                writeln("y[", i, ",", k, ",*,", t, "] = ", TotY[i][k][t]);   

Change OPL writeln to python print:

In [ ]:
TotProdCost    = sum([ProdCost[i,j]*x[i,j,t].x for t in Periods 
                                               for j in Products 
                                               for i in Factories])
TotMovCost_FDC = sum([MovCost_FDC[i]*y[i,2,j,t].x for t in Periods 
                                                  for j in Products 
                                                  for i in Factories]) 
TotMovCost_FC  = sum([MovCost_FC[i]*y[i,3,j,t].x for t in Periods 
                                                 for j in Products 
                                                 for i in Factories]) 
TotMovCost_DCC = sum([MovCost_DCC*z[j,t].x for t in Periods 
                                           for j in Products])                 
TotHoldCost    = sum([HoldCost*q2[j,t].x for t in Periods 
                                         for j in Products])                 
TotTardCost_DC = sum([7*TardCost_DC[j]*v2[j,t].x for t in Periods[:-1] 
                                                 for j in Products])        
TotTardCost_C  = sum([7*TardCost_C[j]*v3[j,t].x for t in Periods[:-1] 
                                                for j in Products])       
TotPenalty     = sum([Penalty*(v2[j,Nperiods].x + v3[j,Nperiods].x) 
                                              for j in Products])

TotY = {(i,k,t) : sum([y[i,k,j,t].x for j in Products]) for i in Factories 
                                                        for k in Stages 
                                                        for t in Periods }
print("Total Production Cost:  ", TotProdCost);
print("Total Move Cost FDC:    ", TotMovCost_FDC);
print("Total Move Cost FC:     ", TotMovCost_FC);
print("Total Move Cost DCC:    ", TotMovCost_DCC);
print("Total Hold Cost:        ", TotHoldCost);
print("Total Tardy Cost DC:    ", TotTardCost_DC);
print("Total Tardy Cost C:     ", TotTardCost_C);
print("Total Penalty:          ", TotPenalty);
   
for i in Factories:
    for k in Stages:
        for t in Periods:
            print("y[", i, ",", k, ",*,", t, "].x = ", TotY[i,k,t])


oplrun scheduling.mod supplydata.dat

    Total Production Cost:  153667
    Total Move Cost FDC:    47033.2
    Total Move Cost FC:     4250
    Total Move Cost DCC:    0
    Total Hold Cost:        0
    Total Tardy Cost DC:    1400000
    Total Tardy Cost C:     1400000
    Total Penalty:          0
    y[1,2,*,1] = 47334
    y[1,2,*,2] = 20000
    y[1,2,*,3] = 50000
    y[1,2,*,4] = 0
    y[1,3,*,1] = 0
    y[1,3,*,2] = 0
    y[1,3,*,3] = 2334
    y[1,3,*,4] = 0
    y[2,2,*,1] = 52666
    y[2,2,*,2] = 25000
    y[2,2,*,3] = 40000
    y[2,2,*,4] = 0
    y[2,3,*,1] = 25000
    y[2,3,*,2] = 15000
    y[2,3,*,3] = 42666
    y[2,3,*,4] = 0

Python Output:

    Total Production Cost:   153667.0
    Total Move Cost FDC:     47033.2
    Total Move Cost FC:      4250.0
    Total Move Cost DCC:     0.0
    Total Hold Cost:         7.27595761418e-13
    Total Tardy Cost DC:     1400000.0
    Total Tardy Cost C:      1400000.0
    Total Penalty:           0.0
    y[ 1 , 2 ,*, 1 ].x =  47334.0
    y[ 1 , 2 ,*, 2 ].x =  20000.0
    y[ 1 , 2 ,*, 3 ].x =  50000.0
    y[ 1 , 2 ,*, 4 ].x =  0.0
    y[ 1 , 3 ,*, 1 ].x =  0.0
    y[ 1 , 3 ,*, 2 ].x =  0.0
    y[ 1 , 3 ,*, 3 ].x =  2334.0
    y[ 1 , 3 ,*, 4 ].x =  0.0
    y[ 2 , 2 ,*, 1 ].x =  52666.0
    y[ 2 , 2 ,*, 2 ].x =  25000.0
    y[ 2 , 2 ,*, 3 ].x =  40000.0
    y[ 2 , 2 ,*, 4 ].x =  0.0
    y[ 2 , 3 ,*, 1 ].x =  25000.0
    y[ 2 , 3 ,*, 2 ].x =  15000.0
    y[ 2 , 3 ,*, 3 ].x =  42666.0
    y[ 2 , 3 ,*, 4 ].x =  0.0
    
This output matches the OPL output.

# End of slides